In [7]:
!pip install tensorflow
!pip install pycocotools
import os
import glob
import numpy as np
import json
import pandas as pd 
from pycocotools.coco import COCO 
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np 
import skimage.io as io
from matplotlib import pyplot as plt
from pprint import pprint
import tensorflow as tf
from tensorflow.keras import layers, models
%matplotlib inline




In [5]:
coco = COCO(r'D:\9517Dataset\archive\turtles-data\data\annotations.json')

loading annotations into memory...
Done (t=3.92s)
creating index...
index created!


In [8]:

import pandas as pd
import numpy as np

# Read the CSV file and initially divide it according to the split_open column
df = pd.read_csv(r'D:\9517Dataset\archive\turtles-data\data\metadata_splits.csv', usecols=['id', 'identity', 'split_open'])

# Extract train, test and valid datasets respectively
train_df = df[df['split_open'] == 'train']
test_df = df[df['split_open'] == 'test']
valid_df = df[df['split_open'] == 'valid']

# Randomly draw 50% of each set
train_half_df = train_df.sample(frac=0.5, random_state=42).reset_index(drop=True)
test_half_df = test_df.sample(frac=0.5, random_state=42).reset_index(drop=True)
valid_half_df = valid_df.sample(frac=0.5, random_state=42).reset_index(drop=True)

# Extract the ID of each collection into a list for later use
id_list_train_half = train_half_df['id'].tolist()
id_list_test_half = test_half_df['id'].tolist()
id_list_valid_half = valid_half_df['id'].tolist()

# Output the number of samples for each subset to ensure correct sampling
print(f"Half Train samples: {len(id_list_train_half)}")
print(f"Half Test samples: {len(id_list_test_half)}")
print(f"Half Valid samples: {len(id_list_valid_half)}")




Half Train samples: 2652
Half Test samples: 1154
Half Valid samples: 559


In [9]:
import json
import pandas as pd

# Read the original JSON file
with open(r'D:/9517Dataset/archive/turtles-data/data/annotations.json', 'r') as f:
    coco_data = json.load(f)

# Get a list of all image ids
all_image_ids = [img['id'] for img in coco_data['images']]

# Filter out image ids belonging to train, test and valid according to the previous ID list
train_image_ids = [img_id for img_id in all_image_ids if img_id in id_list_train_half]
test_image_ids = [img_id for img_id in all_image_ids if img_id in id_list_test_half]
valid_image_ids = [img_id for img_id in all_image_ids if img_id in id_list_valid_half]

# Screen out the corresponding image data and annotation data
train_images = [img for img in coco_data['images'] if img['id'] in train_image_ids]
test_images = [img for img in coco_data['images'] if img['id'] in test_image_ids]
valid_images = [img for img in coco_data['images'] if img['id'] in valid_image_ids]

train_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in train_image_ids]
test_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in test_image_ids]
valid_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in valid_image_ids]

# Create new JSON structure
def create_new_json(images, annotations):
    new_json = {
        "images": images,
        "annotations": annotations,
        "categories": coco_data["categories"]  # Keep the original category information
    }
    return new_json

# Generate a new JSON file
train_json = create_new_json(train_images, train_annotations)
test_json = create_new_json(test_images, test_annotations)
valid_json = create_new_json(valid_images, valid_annotations)

# Save a new JSON file
with open(r'D:/9517Dataset/archive/turtles-data/data/train_annotations.json', 'w') as f:
    json.dump(train_json, f)
    
with open(r'D:/9517Dataset/archive/turtles-data/data/test_annotations.json', 'w') as f:
    json.dump(test_json, f)
    
with open(r'D:/9517Dataset/archive/turtles-data/data/valid_annotations.json', 'w') as f:
    json.dump(valid_json, f)

print("JSON files for train, test, and valid sets have been created successfully.")


JSON files for train, test, and valid sets have been created successfully.


In [13]:
pip install torch torchvision numpy matplotlib opencv-python


Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import json
import os
import numpy as np
import cv2
from pycocotools.coco import COCO

# Step 1: Read the CSV file and divide it according to the split_open column
df = pd.read_csv(r'D:\9517Dataset\archive\turtles-data\data\metadata_splits.csv', usecols=['id', 'identity', 'split_open'])

# Extract train, test and valid datasets respectively, and randomly extract 50% samples
train_half_df = df[df['split_open'] == 'train'].sample(frac=0.5, random_state=42).reset_index(drop=True)
test_half_df = df[df['split_open'] == 'test'].sample(frac=0.5, random_state=42).reset_index(drop=True)
valid_half_df = df[df['split_open'] == 'valid'].sample(frac=0.5, random_state=42).reset_index(drop=True)

# Extract a list of ids for each collection
id_list_train_half = train_half_df['id'].tolist()
id_list_test_half = test_half_df['id'].tolist()
id_list_valid_half = valid_half_df['id'].tolist()

# Output the number of samples for each subset to ensure correct sampling
print(f"Half Train samples: {len(id_list_train_half)}")
print(f"Half Test samples: {len(id_list_test_half)}")
print(f"Half Valid samples: {len(id_list_valid_half)}")

# Step 2: Generate a new JSON file
with open(r'D:/9517Dataset/archive/turtles-data/data/annotations.json', 'r') as f:
    coco_data = json.load(f)

# Filter images and annotate data according to ID list
def filter_data_by_ids(coco_data, id_list):
    images = [img for img in coco_data['images'] if img['id'] in id_list]
    annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in id_list]
    return images, annotations

train_images, train_annotations = filter_data_by_ids(coco_data, id_list_train_half)
test_images, test_annotations = filter_data_by_ids(coco_data, id_list_test_half)
valid_images, valid_annotations = filter_data_by_ids(coco_data, id_list_valid_half)

# Create a new JSON file
def create_new_json(images, annotations, categories):
    return {"images": images, "annotations": annotations, "categories": categories}

# Save the JSON file
with open(r'D:/9517Dataset/archive/turtles-data/data/train_annotations_v2.json', 'w') as f:
    json.dump(create_new_json(train_images, train_annotations, coco_data["categories"]), f)

with open(r'D:/9517Dataset/archive/turtles-data/data/test_annotations_v2.json', 'w') as f:
    json.dump(create_new_json(test_images, test_annotations, coco_data["categories"]), f)

with open(r'D:/9517Dataset/archive/turtles-data/data/valid_annotations_v2.json', 'w') as f:
    json.dump(create_new_json(valid_images, valid_annotations, coco_data["categories"]), f)

print("JSON files for train, test, and valid sets have been created successfully.")

# Step 3: Generate the mask
annotation_path_train = "D:/9517Dataset/archive/turtles-data/data/train_annotations_v2.json"
annotation_path_test = "D:/9517Dataset/archive/turtles-data/data/test_annotations_v2.json"
annotation_path_valid = "D:/9517Dataset/archive/turtles-data/data/valid_annotations_v2.json"
img_dir = "D:/9517Dataset/archive/turtles-data/data"  # Make sure to point to the images folder
mask_dir_train = "D:/9517Dataset/archive/turtles-data/data/masks_train_v2"
mask_dir_test = "D:/9517Dataset/archive/turtles-data/data/masks_test_v2"
mask_dir_valid = "D:/9517Dataset/archive/turtles-data/data/masks_valid_v2"

# Create a mask folder
os.makedirs(mask_dir_train, exist_ok=True)
os.makedirs(mask_dir_test, exist_ok=True)
os.makedirs(mask_dir_valid, exist_ok=True)

# Function that generates the mask
def generate_masks(annotation_path, img_dir, mask_dir):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    coco = COCO(annotation_path)

    for img_info in coco_data['images']:
        file_name = img_info['file_name']
        img_id = img_info['id']
        width, height = img_info['width'], img_info['height']
        
       # Get images path
        image_path = os.path.join(img_dir, file_name)
        
       # Make sure the image file exists
        if not os.path.exists(image_path):
            print(f"Image {file_name} not found at path {image_path}, skipping.")
            continue
        
        # Create an empty color mask image (black background)
        mask = np.zeros((height, width, 3), dtype=np.uint8)
        
       # Gets all annotations for the current image
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        
        for ann in anns:
            # Set a color according to category_id
            mask_part = coco.annToMask(ann)
            if ann['category_id'] == 1:  # Head
                mask[mask_part == 1] = [255, 0, 0]
            elif ann['category_id'] == 2:  # Fins
                mask[mask_part == 1] = [0, 255, 0]
            elif ann['category_id'] == 3:  # Turtle shell
                mask[mask_part == 1] = [0, 0, 255]

        # Save the mask
        mask_subdir = os.path.join(mask_dir, os.path.basename(os.path.dirname(file_name)))  
        os.makedirs(mask_subdir, exist_ok=True)
        mask_path = os.path.join(mask_subdir, f"{os.path.splitext(os.path.basename(file_name))[0]}_mask_v2.png")
        
        # Print the save path and confirm
        print(f"Saving mask at: {mask_path}")
        cv2.imwrite(mask_path, mask)

# Generate masks for training sets, test sets, verification sets
generate_masks(annotation_path_train, img_dir, mask_dir_train)
generate_masks(annotation_path_test, img_dir, mask_dir_test)
generate_masks(annotation_path_valid, img_dir, mask_dir_valid)

print("Masks have been generated for train, test, and valid datasets.")


Half Train samples: 2652
Half Test samples: 1154
Half Valid samples: 559
JSON files for train, test, and valid sets have been created successfully.
loading annotations into memory...
Done (t=1.31s)
creating index...
index created!
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\CAluWEgwPX_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\OiaMESWPyh_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\VMyJozldvy_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\XhbkFIOvZr_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\dLFculJDGP_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\kTIpheVAuj_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_v2\t001\oYBbBMeIrc_mask_v2.png
Saving mask at: D:/9517Dataset/archive/turtles-data/data/masks_train_